# Bring Pandas in Colab to Warp Speed

## **cuDF**
("KOO-dee-eff")

Intro: https://developer.nvidia.com/blog/rapids-cudf-instantly-accelerates-pandas-up-to-50x-on-google-colab/

Product: https://rapids.ai/cudf-pandas/

Sample notebook: https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_colab_demo.ipynb

Documentation: https://docs.rapids.ai/api/cudf/nightly/?_gl=1*99jcxf*_ga*MjIwMjk4NjAxLjE3MTYyMjM3MDg.*_ga_RKXFW6CM42*MTcxNjIzMTY2Ni4yLjAuMTcxNjIzMTY2OC41OC4wLjA.

In [ ]:
# verify you're running an NVIDIA GPU:

!nvidia-smi

Mon May 20 18:56:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%load_ext cudf.pandas

In [ ]:
!pip install -q faker-vehicle
!pip install -q ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.4/162.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.8 MB/s eta 0:00:00


In [ ]:
# show time for each cell execution

%load_ext autotime

time: 884 µs (started: 2024-05-20 18:57:27 +00:00)


In [ ]:
# function to generate fake data
import re
import random

def remove_junk(search_term: str):
  """
  Removes unwanted characters from a search term.

  Args:
    search_term: The search term to be cleaned.

  Returns:
    The cleaned search term with unwanted characters removed.
  """
  # remove blank space
  results = search_term.replace(' ','')
  # remove unecessary characters
  results = re.sub(r'[/\+\-_=~*%$#@!"(){}]', '', results)
  # upper case
  return results.upper()

def create_rows_faker(num: int=1):
  """
  Creates a list of rows with fake data.

  Args:
    num (int): The number of rows to create.

  Returns:
    list: A list of dictionaries containing fake data.
  """
  return_set = []
  for x in range(num):
    randomdata = random.randint(1000,2000)
    model_number = fake.machine_model()
    # if model number is less than 6 characters, add randomdata, adjust as desired
    if len(model_number) < 6:
      model_number += str(randomdata)
    return_set.append({"brand":fake.machine_make(),
                   "model_number":model_number,
                   "model_name":fake.machine_category(),
                   "year":fake.machine_year(),
                   "randomdata":randomdata,
                   "model_search":remove_junk(model_number)
                       })
  return return_set


time: 1.11 ms (started: 2024-05-20 18:57:30 +00:00)


In [ ]:
from faker import Faker
from faker_vehicle import VehicleProvider

fake = Faker()
fake.add_provider(VehicleProvider)

# generate fake data
import pandas as pd

number_of_sample_rows = 5000000
df_faker = pd.DataFrame(create_rows_faker(number_of_sample_rows))

print(df_faker.info())
df_faker.head()


<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 6 columns):
 #   Column        Dtype
---  ------        -----
 0   brand         object
 1   model_number  object
 2   model_name    object
 3   year          object
 4   randomdata    int64
 5   model_search  object
dtypes: int64(1), object(5)
memory usage: 341.5+ MB
None


,brand,model_number,model_name,year,randomdata,model_search
0,Jonyang,RG1300,Hydraulic Excavator,2003,1492,RG1300
1,Challenger,SD75B Padfoot,Pneumatic Roller,2018,1203,SD75BPADFOOT
2,Massey Ferguson,T6.145,Disc,2004,1237,T6.145
3,McCormick,D31EX-21 KOMSTAT II,Combine,2011,1337,D31EX21KOMSTATII
4,New Holland,D65WX-15 PAT,Disc,2013,1951,D65WX15PAT


time: 3min 4s (started: 2024-05-20 18:58:13 +00:00)


In [ ]:
# advanced groupby
(df_faker[['brand','year','model_name']]
.value_counts()
.groupby('brand')
.head(1)
.sort_index()
.reset_index()
)


,brand,year,model_name,count
0,AAA,2000,4WD Tractor,7
1,AGCO,2018,4WD Tractor,214
2,AMCO,2011,4WD Tractor,225
3,ASV,2004,4WD Tractor,69
4,Abbriata,2005,4WD Tractor,7
...,...,...,...,...
182,XCMG,2009,4WD Tractor,7
183,YTO,2014,4WD Tractor,162
184,Yanmar,2018,4WD Tractor,279
185,Zettelmeyer,2011,4WD Tractor,192


time: 2.73 s (started: 2024-05-20 19:02:40 +00:00)


### Supported and Unsupported Operations

Not all Pandas operations are currently supported. Where not supported, CPU will be used with no optimizations.

In [16]:
# supported and unsupported operations

df_faker.count()

brand           5000000
model_number    5000000
model_name      5000000
year            5000000
randomdata      5000000
model_search    5000000
dtype: int64

time: 5.69 s (started: 2024-05-20 19:46:05 +00:00)


In [ ]:
# `axis` not supported currently for `count()`  -- much slower

df_faker.count(axis=1)

0          6
1          6
2          6
3          6
4          6
          ..
4999995    6
4999996    6
4999997    6
4999998    6
4999999    6
Length: 5000000, dtype: int64

time: 11.2 s (started: 2024-05-20 19:03:11 +00:00)


## Profiling Performance

- `%%cudf.pandas.profile`:  per function profile
- `%%cudf.pandas.line_profile`:  per line profile

Note (from cudf_pandas_colab_demo.ipynb):
"If you're running in Colab, the first time you run use the profiler it may take 10+ seconds due to Colab's debugger interacting with the built-in Python function [sys.settrace](https://docs.python.org/3/library/sys.html#sys.settrace) that we use for profiling. For demo purposes, this isn't an issue. Just run the cell again."

In [ ]:
%%cudf.pandas.profile

number_of_sample_rows = 10000
df_small_faker = pd.DataFrame(create_rows_faker(number_of_sample_rows))

# advanced groupby
(df_small_faker[['brand','year','model_name']]
.value_counts()
.groupby('brand')
.head(1)
.sort_index()
.reset_index()
)


,brand,year,model_name,count
0,AAA,2000,Hydraulic Excavator,1
1,AGCO,2005,Hydraulic Excavator,3
2,AMCO,2013,Hydraulic Excavator,3
3,ASV,2019,Mini Excavator,2
4,Agco Sunflower,2001,Vibratory Compactor,1
...,...,...,...,...
172,XCMG,2014,Vibratory Smooth Drum Roller,1
173,YTO,2014,4WD Tractor,3
174,Yanmar,2000,Hydraulic Excavator,2
175,Zettelmeyer,2011,4WD Tractor,2


time: 6.52 s (started: 2024-05-20 19:26:45 +00:00)


                                                                                                            
                                         Total time elapsed: 6.821 seconds                                  
                                       43 GPU function calls in 4.586 seconds                               
                                       13 CPU function calls in 0.040 seconds                               
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame              │ 2          │ 4.354       │ 2.177       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__getitem__  │ 1          │ 0.008       │ 0.008       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.value_counts │ 1          │ 0.051       │ 0.051       │ 0          │ 0.000       │ 0.000       │
│ Series.groupby         │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ SeriesGroupBy.head     │ 1          │ 0.033       │ 0.033       │ 0          │ 0.000       │ 0.000       │
│ Series.sort_index      │ 1          │ 0.011       │ 0.011       │ 0          │ 0.000       │ 0.000       │
│ Series.reset_index     │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__repr__     │ 1          │ 0.064       │ 0.064       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__len__      │ 3          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.copy         │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Index.__len__          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.items        │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Series.nunique         │ 6          │ 0.004       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.__len__         │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Series.notna           │ 4          │ 0.003       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.__getitem__     │ 5          │ 0.019       │ 0.004       │ 0          │ 0.000       │ 0.000       │
│ Series.unique          │ 1          │ 0.002       │ 0.002       │ 3          │ 0.016       │ 0.005       │
│ ndarray.__len__        │ 1          │ 0.000       │ 0.000       │ 3          │ 0.002       │ 0.001       │
│ Series                 │ 1          │ 0.001       │ 0.001       │ 3          │ 0.003       │ 0.001       │
│ Series.sample          │ 4          │ 0.021       │ 0.005       │ 0          │ 0.000       │ 0.000       │
│ Series.tolist          │ 0          │ 0.000       │ 0.000       │ 4          │ 0.020       │ 0.005       │
│ to_datetime            │ 1          │ 0.009       │ 0.009       │ 0          │ 0.000       │ 0.000       │
│ Series.std             │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.min             │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.max             │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- Series.unique
- ndarray.__len__
- Series
- Series.tolist

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=995206;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

time: 6.69 s (started: 2024-05-20 19:26:45 +00:00)


In [ ]:
%%cudf.pandas.line_profile

number_of_sample_rows = 10000
df_small_faker = pd.DataFrame(create_rows_faker(number_of_sample_rows))

# advanced groupby
(df_small_faker[['brand','year','model_name']]
.value_counts()
.groupby('brand')
.head(1)
.sort_index()
.reset_index()
)


                                                                                                                   
                                             Total time elapsed: 8.004 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     number_of_sample_rows = 10000                                        │             │             │
│          │                                                                          │             │             │
│ 3        │     df_small_faker = pd.DataFrame(create_rows_faker(number_of_sample_ro… │ 4.654826109 │             │
│          │                                                                          │             │             │
│ 6        │     (df_small_faker[['brand','year','model_name']]                       │ 0.004749591 │             │
│          │                                                                          │             │             │
│ 7        │     .value_counts()                                                      │ 0.052689646 │             │
│          │                                                                          │             │             │
│ 8        │     .groupby('brand')                                                    │ 0.000830933 │             │
│          │                                                                          │             │             │
│ 9        │     .head(1)                                                             │ 0.034084536 │             │
│          │                                                                          │             │             │
│ 10       │     .sort_index()                                                        │ 0.012322731 │             │
│          │                                                                          │             │             │
│ 11       │     .reset_index()                                                       │ 0.001149383 │             │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

time: 8.06 s (started: 2024-05-20 19:28:42 +00:00)
time: 8.06 s (started: 2024-05-20 19:28:42 +00:00)


## GPU without cuDF

In [ ]:
# force restart
get_ipython().kernel.do_shutdown(restart=True)

In [1]:
!pip install -q faker-vehicle
!pip install -q ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.4/162.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00


In [2]:
# show time for each cell execution

%load_ext autotime

time: 366 µs (started: 2024-05-20 19:55:35 +00:00)


In [3]:
# function to generate fake data
import re
import random

def remove_junk(search_term: str):
  """
  Removes unwanted characters from a search term.

  Args:
    search_term: The search term to be cleaned.

  Returns:
    The cleaned search term with unwanted characters removed.
  """
  # remove blank space
  results = search_term.replace(' ','')
  # remove unecessary characters
  results = re.sub(r'[/\+\-_=~*%$#@!"(){}]', '', results)
  # upper case
  return results.upper()

def create_rows_faker(num: int=1):
  """
  Creates a list of rows with fake data.

  Args:
    num (int): The number of rows to create.

  Returns:
    list: A list of dictionaries containing fake data.
  """
  return_set = []
  for x in range(num):
    randomdata = random.randint(1000,2000)
    model_number = fake.machine_model()
    # if model number is less than 6 characters, add randomdata, adjust as desired
    if len(model_number) < 6:
      model_number += str(randomdata)
    return_set.append({"brand":fake.machine_make(),
                   "model_number":model_number,
                   "model_name":fake.machine_category(),
                   "year":fake.machine_year(),
                   "randomdata":randomdata,
                   "model_search":remove_junk(model_number)
                       })
  return return_set


time: 1.15 ms (started: 2024-05-20 19:55:38 +00:00)


In [4]:
from faker import Faker
from faker_vehicle import VehicleProvider

fake = Faker()
fake.add_provider(VehicleProvider)

# generate fake data
import pandas as pd

number_of_sample_rows = 5000000
df_faker = pd.DataFrame(create_rows_faker(number_of_sample_rows))

print(df_faker.info())
df_faker.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   brand         object
 1   model_number  object
 2   model_name    object
 3   year          object
 4   randomdata    int64 
 5   model_search  object
dtypes: int64(1), object(5)
memory usage: 228.9+ MB
None


,brand,model_number,model_name,year,randomdata,model_search
0,Bomag,WA900-3E0,4WD Tractor,2012,1006,WA9003E0
1,Krause,PF2001566,Mini Excavator,2006,1566,PF2001566
2,Massey Ferguson,314D CR/314D LCR,Skid Steer Loader,2004,1259,314DCR314DLCR
3,Komatsu,WA450-6 Waste Handler,Skidder,2015,1641,WA4506WASTEHANDLER
4,Ezee-On,PC600LC-6,Wheel Loader,2010,1096,PC600LC6


time: 1min 20s (started: 2024-05-20 19:55:44 +00:00)


In [7]:
type(df_faker)

pandas.core.frame.DataFrame

time: 227 ms (started: 2024-05-20 19:58:06 +00:00)


In [8]:
# advanced groupby
(df_faker[['brand','year','model_name']]
.value_counts()
.groupby('brand')
.head(1)
.sort_index()
.reset_index()
)


,brand,year,model_name,count
0,AAA,2011,Hydraulic Excavator,8
1,AGCO,2007,4WD Tractor,203
2,AMCO,2011,4WD Tractor,214
3,ASV,2015,4WD Tractor,76
4,Abbriata,2017,Wheel Loader,7
...,...,...,...,...
182,XCMG,2015,Wheel Loader,9
183,YTO,2010,4WD Tractor,155
184,Yanmar,2011,4WD Tractor,276
185,Zettelmeyer,2010,4WD Tractor,208


time: 1.82 s (started: 2024-05-20 19:58:19 +00:00)
